# dev

In [1]:
# constants
bucket = "k9-kedro-on-aws"
tbl = "transactions"

## Check AWS conf

Setup the credentials in your preferred manner.

In [2]:
%%bash
aws sts get-caller-identity

{
    "UserId": "AIDARAMT7QKYJ6DR7ITQU",
    "Account": "069566694064",
    "Arn": "arn:aws:iam::069566694064:user/k9"
}


In [3]:
%%bash -s $bucket
aws s3 ls $1

                           PRE transactions/


## Check S3 connection on pyspark

Note that we will need two additional `jars` in order to access AWS S3 from Spark:

- [hadoop-aws](https://mvnrepository.com/artifact/org.apache.hadoop/hadoop-aws)
- [aws-java-sdk-bundle](https://mvnrepository.com/artifact/com.amazonaws/aws-java-sdk-bundle)

Make sure they are present in your `SPARK_HOME` jars folder.
In the case of a `pip install pyspark`, the folder should be something like `<your-python-lib-path>/site-packages/pyspark/jars`.

In [4]:
from pyspark.sql import SparkSession

# spark configuration is handled by conf/base/spark.yml
# refer to the src/hooks.py for details
spark = SparkSession.builder.getOrCreate()

In [5]:
spark.read.parquet(f"s3a://{bucket}/{tbl}").printSchema()

23/01/08 19:23:04 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


23/01/08 19:23:08 WARN DataSource: Found duplicate column(s) in the data schema and the partition schema: `date`
root
 |-- gas: long (nullable = true)
 |-- hash: string (nullable = true)
 |-- input: string (nullable = true)
 |-- nonce: long (nullable = true)
 |-- value: double (nullable = true)
 |-- block_number: long (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- transaction_index: long (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- gas_price: long (nullable = true)
 |-- receipt_cumulative_gas_used: long (nullable = true)
 |-- receipt_gas_used: long (nullable = true)
 |-- receipt_contract_address: string (nullable = true)
 |-- receipt_status: long (nullable = true)
 |-- receipt_effective_gas_price: long (nullable = true)
 |-- transaction_type: long (nullable = true)
 |-- max_fee_per_gas: long (nullable = true)
 |-- max_priority_fee_per_gas: long (nullable = true)
 |-- block_timestamp: timestamp (nullable =

## Test kedro catalog

In [6]:
catalog.list()

['transactions', 'parameters', 'params:train_fraction', 'params:random_state', 'params:target_column']

In [1]:
transactions = catalog.load("transactions")

[01/08/23 19:24:06] INFO     Loading data from 'transactions' (AthenaQueryDataSet)...           ]8;id=22244;file:///Users/k9/.pyenv/versions/3.9.11/envs/kedro-on-aws/lib/python3.9/site-packages/kedro/io/data_catalog.py\data_catalog.py]8;;\:]8;id=662374;file:///Users/k9/.pyenv/versions/3.9.11/envs/kedro-on-aws/lib/python3.9/site-packages/kedro/io/data_catalog.py#343\343]8;;\

                    INFO     Attempt to create external table temp_tbl_b98165941e with Athena  ]8;id=930635;file:///Users/k9/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py\athena_dataset.py]8;;\:]8;id=359149;file:///Users/k9/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py#81\81]8;;\
                             CTAS query.                                                                           

[01/08/23 19:24:11] INFO     Athena query output files saved to:                              ]8;id=946790;file:///Users/k9/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py\athena_dataset.py]8;;\:]8;id=617971;file:///Users/k9/kedro-on-aws/src/iris/extras/datasets/athena_dataset.py#111\111]8;;\
                             s3://k9-kedro-on-aws/athena-query-outputs/tables/270e3c92-3bc1-4                      
                             f7e-8939-e39ebcb9311e                                                                 

23/01/08 19:24:11 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties


In [2]:
transactions.printSchema()

root
 |-- date: string (nullable = true)
 |-- hash: string (nullable = true)
 |-- nonce: long (nullable = true)
 |-- transaction_index: long (nullable = true)
 |-- from_address: string (nullable = true)
 |-- to_address: string (nullable = true)
 |-- value: double (nullable = true)
 |-- gas: long (nullable = true)
 |-- gas_price: long (nullable = true)
 |-- input: string (nullable = true)
 |-- receipt_cumulative_gas_used: long (nullable = true)
 |-- receipt_gas_used: long (nullable = true)
 |-- receipt_contract_address: string (nullable = true)
 |-- receipt_status: long (nullable = true)
 |-- block_timestamp: timestamp (nullable = true)
 |-- block_number: long (nullable = true)
 |-- block_hash: string (nullable = true)
 |-- max_fee_per_gas: long (nullable = true)
 |-- max_priority_fee_per_gas: long (nullable = true)
 |-- transaction_type: long (nullable = true)
 |-- receipt_effective_gas_price: long (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = t

In [3]:
transactions.count()

100